### wps_climdex_sdii

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s [climdex.sdii](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=24) function

**climdex.sdii** computes the climdex index SDII, or Simple Precipitation Intensity Index. This isdefined as the sum of precipitation in wet days (days with preciptitation over 1mm) during the yeardivided by the number of wet days in the year.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from importlib.resources import files
from tempfile import NamedTemporaryFile

from wps_tools.R import rda_to_vector, construct_r_out, test_rda_output
from wps_tools.testing import get_target_url

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_sdii?

Signature:
quail.climdex_sdii(
    climdex_input=None,
    loglevel='INFO',
    output_file='output.rda',
    output_formats=None,
)
Docstring:
Defined as the sum of precipitation in wet days (days with preciptitation over 1mm) during the year divided by the number of wet days in the year.

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/github/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-12>
Type:      method


#### Run wps_climdex_sdii Process with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="sdii_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_sdii(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),
            output_file=output_file.name,
        )
sdii_url = output.get()[0]

You can also use rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="sdii_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_sdii(
            climdex_input=(files("tests") / "data/climdexInput.rds").resolve(),
            output_file=output_file.name,
        )
sdii_url = output.get()[0]

And with multiple input

In [8]:
climdex_inputs = [
    (files("tests") / "data/climdexInput.rds").resolve(),
    (files("tests") / "data/climdexInput.rda").resolve(),
    (files("tests") / "data/climdex_input_multiple.rda").resolve(),
]
with NamedTemporaryFile(suffix=".rda", prefix="sdii_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_sdii(
            climdex_input=climdex_inputs,
            output_file=output_file.name,
        )

In [9]:
# NBVAL_IGNORE_OUTPUT
construct_r_out(output.get())

[[R object with classes: ('numeric',) mapped to:
  [     nan, 7.175893,      nan,      nan, ..., 6.025490,      nan, 7.321192,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 7.175893,      nan,      nan, ..., 6.025490,      nan, 7.321192,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 7.175893,      nan,      nan, ..., 6.025490,      nan, 7.321192,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 7.175893,      nan,      nan, ..., 6.025490,      nan, 7.321192,      nan]]]

Access the output with **get_robjects()** or **auto_construct_outputs()** from **wps_tools.output_handling**

In [10]:
# NBVAL_IGNORE_OUTPUT
sdii = rda_to_vector(sdii_url, "sdii1_ci")
# use print() to see whole vector
print(sdii)

    1959     1960     1961     1962     1963     1964     1965     1966 
      NA 7.175893       NA       NA       NA 6.052414 7.669298 7.074312 
    1967     1968     1969     1970     1971     1972     1973     1974 
8.576190 7.460145 6.150943 7.014545 7.868702 8.316667 5.878070 7.509565 
    1975     1976     1977     1978     1979     1980     1981     1982 
9.175229       NA       NA 4.683200 5.818699 6.355128 6.175510       NA 
    1983     1984     1985     1986     1987     1988     1989     1990 
7.610000       NA       NA       NA       NA       NA 7.353846 9.232143 
    1991     1992     1993     1994     1995     1996     1997     1998 
      NA 6.755932       NA       NA 7.675974       NA 7.125989 7.438776 
    1999     2000     2001     2002     2003     2004 
7.770552 4.912879 6.025490       NA 7.321192       NA 



#### Test output against expected output

In [11]:
test_rda_output(
    sdii_url, "sdii1_ci", "expected_sdii.rda", "expected_sdii"
    )